# Ray RLlib - Simple Multi-Armed Bandits Example

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)



A very simple contextual bandit example with 3 arms

In [1]:
import gym
from gym.spaces import Discrete, Box
import numpy as np
import random
from ray import tune
import time

In [2]:
class SimpleContextualBandit (gym.Env):
    def __init__ (self, config=None):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=-1., high=1., shape=(2, ), dtype=np.float64)
        self.cur_context = None

    def reset (self):
        self.cur_context = random.choice([-1., 1.])
        return np.array([self.cur_context, -self.cur_context])

    def step (self, action):
        rewards_for_context = {
            -1.: [-10, 0, 10],
            1.: [10, 0, -10],
        }
        
        reward = rewards_for_context[self.cur_context][action]
        
        return (np.array([-self.cur_context, self.cur_context]), reward, True,
                {
                    "regret": 10 - reward
                })

In [7]:
bandit = SimpleContextualBandit()
bandit.reset()

array([-1.,  1.])

In [8]:
for i in range(10):
    obs = bandit.step(bandit.action_space.sample())
    print(obs)

(array([ 1., -1.]), -10, True, {'regret': 20})
(array([ 1., -1.]), 0, True, {'regret': 10})
(array([ 1., -1.]), 0, True, {'regret': 10})
(array([ 1., -1.]), 10, True, {'regret': 0})
(array([ 1., -1.]), -10, True, {'regret': 20})
(array([ 1., -1.]), 10, True, {'regret': 0})
(array([ 1., -1.]), -10, True, {'regret': 20})
(array([ 1., -1.]), 10, True, {'regret': 0})
(array([ 1., -1.]), 10, True, {'regret': 0})
(array([ 1., -1.]), -10, True, {'regret': 20})


In [9]:
stop = {
    "training_iteration": 200,
    "timesteps_total": 100000,
    "episode_reward_mean": 10.0,
}

config = {
    "env": SimpleContextualBandit,
}

In [ ]:
start_time = time.time()

analysis = tune.run("contrib/LinUCB", config=config, stop=stop)

print("The trials took", time.time() - start_time, "seconds\n")